Import Libraries

In [31]:
import pandas as pd  # for handling data frames.
from jellyfish import soundex, metaphone, levenshtein_distance, jaro_winkler_similarity #for phonetic encoding and string similarity functions.
from collections import defaultdict # for creating dictionaries with default values.
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor #for parallel processing.
import itertools #for efficient looping and combinations.

Preprocess the Name

In [32]:
def preprocess_name(name: str) -> str:
    return name.strip().lower() if isinstance(name, str) else ''

#strip() removes leading and trailing spaces.
#lower() converts the string to lowercase.

Calculate Similarity Score

In [33]:
def get_similarity_score(name1, name2):
    name1, name2 = preprocess_name(name1), preprocess_name(name2) #Preprocess both names.

    #Phonetic codes
    #Compute phonetic scores using soundex and metaphone.
    soundex_score = soundex(name1) == soundex(name2)
    metaphone_score = metaphone(name1) == metaphone(name2)

    #String similarity scores
    #Compute string similarity scores using levenshtein_distance and jaro_winkler_similarity
    levenshtein_score = levenshtein_distance(name1, name2)
    jaro_winkler_score = jaro_winkler_similarity(name1, name2)

    #Combine these scores with weights to get a final similarity score.
    score = (soundex_score * 0.2 +
             metaphone_score * 0.2 +
             (1 - levenshtein_score / max(len(name1), len(name2))) * 0.3 +
             jaro_winkler_score * 0.3)

    return score

In [34]:
# Load the CSV file
org_members_data = pd.read_csv('org_members.csv')

In [35]:
# Preprocess the data: strip spaces and convert to lower case
org_members_data['name'] = org_members_data['name'].apply(preprocess_name)

Block Names:

This function groups names by their first letter to reduce the number of comparisons

In [36]:
#Blocking function based on the first letter of the name
#Create a dictionary where keys are the first letters, and values are lists of names starting with that letter
def block_names(df: pd.DataFrame) -> dict[str, list[str]]:
    blocks = defaultdict(list)
    for _, row in df.iterrows():
        first_letter = row['name'][0] if row['name'] else ''
        blocks[first_letter].append(row['name'])
    return blocks

Process Chunk of Names:

This function takes a chunk of names and compares each pair to see if they are similar based on a threshold. If they are, it records them as potential duplicates.

In [37]:
#Define a function to process a chunk of names:
#Calculate similarity scores for each pair of names.
#If the score exceeds a threshold, add them as potential duplicates.
#Ensure no duplicate pairs by using a set.
def process_chunk(chunk: list[str], threshold: float = 0.8) -> dict[str, list[str]]:
    potential_duplicates = defaultdict(list)
    seen_pairs = set()

    for name1, name2 in itertools.combinations(chunk, 2):
        # Ensure name1 is less than name2 to avoid duplicates and unnecessary repetitions
        if name1 > name2:
            name1, name2 = name2, name1

        pair = (name1, name2)

        if pair not in seen_pairs:
            score = get_similarity_score(name1, name2)
            if score > threshold and score < 1.00:
                potential_duplicates[name1].append(f"{name2} ({score:.2f})")
                potential_duplicates[name2].append(f"{name1} ({score:.2f})")
            seen_pairs.add(pair)

    return potential_duplicates

In [38]:
#Create batches based on the blocked names
blocked_org_members = block_names(org_members_data)

Process Blocks with Threads:

This function processes names in parallel using multiple threads to speed up the comparison process.

In [39]:
#Define a function to process blocks of names using threads:
#Divide names into chunks.
#Process each chunk in parallel using ThreadPoolExecutor.
#Combine results from all threads.

#Function to process blocks using threads
def process_block_with_threads(names: list[str], threshold: float = 0.8, chunk_size: int = 1000) -> dict[str, list[str]]:
    potential_duplicates = defaultdict(list)
    with ThreadPoolExecutor() as executor:
        futures = []
        for i in range(0, len(names), chunk_size):
            chunk = names[i:i + chunk_size]
            futures.append(executor.submit(process_chunk, chunk, threshold))

        for future in futures:
            result = future.result()
            for key, value in result.items():
                potential_duplicates[key].extend(value)

    return potential_duplicates

Process All Blocks in Parallel:

This function processes all blocks in parallel using multiple processes to further speed up the overall computation.

In [40]:
#Process each block in parallel using ProcessPoolExecutor
#Submit blocks for processing, Collect and combine results

all_potential_duplicates = defaultdict(list)
with ProcessPoolExecutor() as executor:
    futures = []
    for names in blocked_org_members.values():
        futures.append(executor.submit(process_block_with_threads, names))

    for future in futures:
        result = future.result()
        for key, value in result.items():
            all_potential_duplicates[key].extend(value)


Consolidate Similar Names into Clusters:

This part combines similar names into clusters and ensures each name is only processed once.

In [41]:
#Consolidate similar names into clusters
#Create clusters of similar names.
#Mark names as visited to avoid processing them multiple times.

clusters = []
visited = set()

for key, similars in all_potential_duplicates.items():
    if key not in visited:
        cluster = [key] + similars
        unique_cluster = sorted(set(cluster), key=str.lower)  # Remove duplicates and sort alphabetically
        visited.update(preprocess_name(name.split(' (')[0]) for name in unique_cluster)  # Update with base name
        clusters.append(unique_cluster)


In [42]:
# Sort clusters alphabetically by name
clusters.sort(key=lambda x: x[0])

# Create DataFrame for clusters
clustered_df = pd.DataFrame(clusters)

In [43]:
# Rename columns
max_cols = clustered_df.shape[1]
column_names = ['name'] + [f'duplicate_{i}' for i in range(1, max_cols)]
clustered_df.columns = column_names

In [44]:
# Save clusters to a CSV file
clustered_df.to_csv('DUPLICATE_NAMES.csv', index=False)